In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files



import requests # library to handle requests


from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### Upload the File with Mississauga Locations and Neighborhoods

In [3]:
path = 'Mississagua Locations_updated.csv'

df = pd.read_csv(path)
df

,Postal Code,Neighborhood,Latitude,Longitude
0,L4Y,"West Applewood, West Dixie, NW Lakeview",43.607600,-79.59970
1,L5M,"Churchill Meadows, Central Erin Mills, South S...",43.561800,-79.72320
2,L4X,"East Applewood, East Dixie, NE Lakeview",43.618838,-79.58343
3,L5A,"Mississauga Valleys, East Cooksville",43.589600,-79.61090
4,L5E,Central Lakeview,43.585500,-79.56600
5,L5B,"West Cooksville, Fairview, City Centre, East C...",43.582100,-79.63340
6,L5G,"SW Lakeview, Mineola, East Port Credit",43.567500,-79.57720
7,L4W,"Matheson, East Rathwood",43.647600,-79.62210
8,L4Z,"West Rathwood, East Hurontario, SE Gateway",43.622200,-79.65580
9,L5R,"West Hurontario, SW Gateway",43.614800,-79.67830


### Install Geopy and Folium Libraries

In [4]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.1
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.11.8  |       ha878542_0         145 KB  conda-forge
    certifi-2020.11.8          |   py36h5fab9bb_0         150 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         392 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch

In [5]:
!pip install folium
import folium

### Map of Mississauga with superimposed neighborhoods

In [6]:
address = 'Mississauga, Canada'

geolocator = Nominatim(user_agent="Miss_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mississauga, Canada {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mississauga, Canada 43.590338, -79.645729.


In [7]:
# create map of Toronto using latitude and longitude values
map_miss = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_miss)  
    
map_miss

#### Get the Rental Car venue from Foursquare Location Provider

In [8]:
CLIENT_ID = '5H4EEPLPOSTAG41MYOR4JGKJ1DBRHSIKTKAJP0EH5RHQ3CTU' # your Foursquare ID
CLIENT_SECRET = '2OVKFUM3FA302LVTO4ANLNPBTJ0QA3AJH030EJVT4WSOPT3H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5H4EEPLPOSTAG41MYOR4JGKJ1DBRHSIKTKAJP0EH5RHQ3CTU
CLIENT_SECRET:2OVKFUM3FA302LVTO4ANLNPBTJ0QA3AJH030EJVT4WSOPT3H


#### Setup query for 'car rental'

In [45]:
radius = 10000
LIMIT = 100

venues1 = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=car rental".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each venue
    for venue in results:
        venues1.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
                 

In [46]:
venues1_df = pd.DataFrame(venues1)

# define the column names
venues1_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'Venue Name', 'Venue Latitude', 'Venue Longitude', 'Venue Category']

print(venues1_df.shape)
venues1_df.head()

(881, 7)


,Neighborhood,Latitude,Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,Enterprise Rent-A-Car,43.683041,-79.611507,Rental Car Location
1,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,Hertz,43.682373,-79.611094,Rental Car Location
2,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,Avis Car Rental,43.683010,-79.612779,Rental Car Location
3,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,National Car Rental,43.686466,-79.619245,Rental Car Location
4,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,Hertz Car Rental,43.682549,-79.612010,Rental Car Location


In [47]:
#Total number of car rental companies grouped by neighborhood
print('There are {} uniques categories.'.format(len(venues1_df['Venue Name'].unique())))

There are 30 uniques categories.


In [48]:
#Total number of car rental companies grouped by neighborhood
print('There are {} uniques categories.'.format(len(venues1_df['Venue Category'].unique())))

There are 2 uniques categories.


In [50]:
v2 = venues1_df.drop(['Latitude', 'Longitude', 'Venue Name', 'Venue Latitude', 'Venue Longitude'], axis=1)
v2.head()

,Neighborhood,Venue Category
0,"West Applewood, West Dixie, NW Lakeview",Rental Car Location
1,"West Applewood, West Dixie, NW Lakeview",Rental Car Location
2,"West Applewood, West Dixie, NW Lakeview",Rental Car Location
3,"West Applewood, West Dixie, NW Lakeview",Rental Car Location
4,"West Applewood, West Dixie, NW Lakeview",Rental Car Location


In [51]:
v2 =v2.groupby('Neighborhood').count().reset_index()
v2.head()

,Neighborhood,Venue Category
0,Central Lakeview,32
1,"Churchill Meadows, Central Erin Mills, South S...",24
2,Clarkson,21
3,"Clarkson, Erin Mills",26
4,"Courtney Park, East Gateway",61


In [52]:
#Total number of car rental companies grouped by neighborhood
print('There are {} uniques categories.'.format(len(v2['Venue Category'].unique())))

There are 20 uniques categories.


In [53]:
venues1_df_new = venues1_df.drop(['Venue Category'], axis=1)
venues1_df_new.head()

,Neighborhood,Latitude,Longitude,Venue Name,Venue Latitude,Venue Longitude
0,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,Enterprise Rent-A-Car,43.683041,-79.611507
1,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,Hertz,43.682373,-79.611094
2,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,Avis Car Rental,43.683010,-79.612779
3,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,National Car Rental,43.686466,-79.619245
4,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,Hertz Car Rental,43.682549,-79.612010


### Modeling - KMeans Clustering

In [54]:
# set number of clusters
kclusters = 4

X= v2.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(X)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 3, 3, 0, 0, 1, 3, 2, 2], dtype=int32)

In [55]:
# add clustering labels
v2.insert(0, 'Cluster Labels', kmeans.labels_)

v2_merged = venues1_df_new 

# merge scarborough_grouped with scarborough_data to add latitude/longitude for each neighborhood
v2_merged = v2_merged.join(v2.set_index('Neighborhood'), on='Neighborhood')

v2_merged.head()

,Neighborhood,Latitude,Longitude,Venue Name,Venue Latitude,Venue Longitude,Cluster Labels,Venue Category
0,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,Enterprise Rent-A-Car,43.683041,-79.611507,2,49
1,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,Hertz,43.682373,-79.611094,2,49
2,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,Avis Car Rental,43.683010,-79.612779,2,49
3,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,National Car Rental,43.686466,-79.619245,2,49
4,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,Hertz Car Rental,43.682549,-79.612010,2,49


## Cluster Map

In [56]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(v2_merged['Latitude'], v2_merged['Longitude'], v2_merged['Neighborhood'], v2_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow [cluster-1],
        fill=True,
        fill_color=rainbow [cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 0 Red

In [57]:
# Cluster 0 Red
Cluster_0 = v2_merged.loc[v2_merged['Cluster Labels'] == 0, v2_merged.columns[[0] + list(range(4, v2_merged.shape[1]))]]

print (Cluster_0['Venue Category'].unique())
Cluster_0.head()

#53 -61 no of rental compnies

[53 58 61 55]


,Neighborhood,Venue Latitude,Venue Longitude,Cluster Labels,Venue Category
73,"East Applewood, East Dixie, NE Lakeview",43.683041,-79.611507,0,53
74,"East Applewood, East Dixie, NE Lakeview",43.682373,-79.611094,0,53
75,"East Applewood, East Dixie, NE Lakeview",43.683010,-79.612779,0,53
76,"East Applewood, East Dixie, NE Lakeview",43.686466,-79.619245,0,53
77,"East Applewood, East Dixie, NE Lakeview",43.682549,-79.612010,0,53


In [58]:
Cluster_0_grouped = Cluster_0.groupby('Neighborhood').count().reset_index()
Cluster_0_grouped

,Neighborhood,Venue Latitude,Venue Longitude,Cluster Labels,Venue Category
0,"Courtney Park, East Gateway",61,61,61,61
1,"East Applewood, East Dixie, NE Lakeview",53,53,53,53
2,"Matheson, East Rathwood",58,58,58,58
3,Toronto Pearson International Airport,55,55,55,55


### Cluster 1 Purple

In [59]:
#Cluster 1 Purple
Cluster_1 =v2_merged.loc[v2_merged['Cluster Labels'] == 1, v2_merged.columns[[0] + list(range(4, v2_merged.shape[1]))]]

print (Cluster_1['Venue Category'].unique())
Cluster_1.head()

# 29 - 37 no. renatl companies

[37 32 30 33 29]


,Neighborhood,Venue Latitude,Venue Longitude,Cluster Labels,Venue Category
126,"Mississauga Valleys, East Cooksville",43.588777,-79.607382,1,37
127,"Mississauga Valleys, East Cooksville",43.586025,-79.608886,1,37
128,"Mississauga Valleys, East Cooksville",43.584694,-79.611418,1,37
129,"Mississauga Valleys, East Cooksville",43.592308,-79.633863,1,37
130,"Mississauga Valleys, East Cooksville",43.592470,-79.634890,1,37


In [60]:
Cluster_1_grouped = Cluster_1.groupby('Neighborhood').count().reset_index()
Cluster_1_grouped

,Neighborhood,Venue Latitude,Venue Longitude,Cluster Labels,Venue Category
0,Central Lakeview,32,32,32,32
1,East Credit,29,29,29,29
2,"Mississauga Valleys, East Cooksville",37,37,37,37
3,"SW Lakeview, Mineola, East Port Credit",30,30,30,30
4,"West Cooksville, Fairview, City Centre, East C...",37,37,37,37
5,"West Creditview, Mavis, Erindale",33,33,33,33


### Cluster 2 Blue

In [61]:
#Cluster 2  Blue
Cluster_2 = v2_merged.loc[v2_merged['Cluster Labels'] == 2, v2_merged.columns[[0] + list(range(4, v2_merged.shape[1]))]]

print (Cluster_2['Venue Category'].unique())
Cluster_2.head()

# 43 to 49

[49 47 44 48 46 43]


,Neighborhood,Venue Latitude,Venue Longitude,Cluster Labels,Venue Category
0,"West Applewood, West Dixie, NW Lakeview",43.683041,-79.611507,2,49
1,"West Applewood, West Dixie, NW Lakeview",43.682373,-79.611094,2,49
2,"West Applewood, West Dixie, NW Lakeview",43.683010,-79.612779,2,49
3,"West Applewood, West Dixie, NW Lakeview",43.686466,-79.619245,2,49
4,"West Applewood, West Dixie, NW Lakeview",43.682549,-79.612010,2,49


In [62]:
Cluster_2_grouped = Cluster_2.groupby('Neighborhood').count().reset_index()
Cluster_2_grouped

,Neighborhood,Venue Latitude,Venue Longitude,Cluster Labels,Venue Category
0,"Etobicoke, Malton, Streetsville, Toronto Pears...",43,43,43,43
1,"Etobicoke, Malton, Toronto Pearson Internation...",43,43,43,43
2,Malton,48,48,48,48
3,Meadowvale Village,46,46,46,46
4,"West Applewood, West Dixie, NW Lakeview",49,49,49,49
5,"West Hurontario, SW Gateway",44,44,44,44
6,"West Rathwood, East Hurontario, SE Gateway",47,47,47,47


### Cluster 3 Yellow

In [63]:
#Cluster 3 Yellow
Cluster_3 =v2_merged.loc[v2_merged['Cluster Labels'] == 3, v2_merged.columns[[0] + list(range(4, v2_merged.shape[1]))]]

print (Cluster_3['Venue Category'].unique())
Cluster_3.head()

# 17 - 27

[24 21 17 26 27]


,Neighborhood,Venue Latitude,Venue Longitude,Cluster Labels,Venue Category
49,"Churchill Meadows, Central Erin Mills, South S...",43.585101,-79.741007,3,24
50,"Churchill Meadows, Central Erin Mills, South S...",43.589062,-79.739822,3,24
51,"Churchill Meadows, Central Erin Mills, South S...",43.592556,-79.727840,3,24
52,"Churchill Meadows, Central Erin Mills, South S...",43.591560,-79.741431,3,24
53,"Churchill Meadows, Central Erin Mills, South S...",43.595236,-79.731626,3,24


In [64]:
Cluster_3_grouped = Cluster_3.groupby('Neighborhood').count().reset_index()
Cluster_3_grouped

,Neighborhood,Venue Latitude,Venue Longitude,Cluster Labels,Venue Category
0,"Churchill Meadows, Central Erin Mills, South S...",24,24,24,24
1,Clarkson,21,21,21,21
2,"Clarkson, Erin Mills",26,26,26,26
3,Erin Mills,27,27,27,27
4,"Lisgar, Meadowvale",17,17,17,17
5,"West Port Credit, Lorne Park, East Sheridan",21,21,21,21


### Analyze Rental companies in each neighborhood

In [70]:
# one hot encoding
miss_onehot = pd.get_dummies(venues1_df[['Venue Name']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
miss_onehot['Neighborhood'] = venues1_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [miss_onehot.columns[-1]] + list(miss_onehot.columns[:-1])
miss_onehot = miss_onehot[fixed_columns]

miss_onehot.head()

,Neighborhood,Alamo Car Rental,Alamo Rent A Car,Auto Loan Solutions,Avis Car Rental,Budget Car & Truck Rental,Budget Car Rental (Budget Car & Truck Rental),Budget Truck Rental,Denny's,Discount Car & Truck Rentals,Discount Car and Truck Rentals,Dollar,Dollar Car Rental,Dollar Rent A Car,E-Z Rent-A-Car,Enterprise Rent-A-Car,Enterprise Rent-a-Car,Hertz,Hertz Car Rental,Kal Tire,National Car Rental,Payless Car Rental,Routes Car & Truck Rental,Serena Motors Ltd,Thrifty Car Rental,Toronto Airport Limo Link,U-Haul Moving & Storage at Rexdale Blvd,U-Haul at Weston Rd,Uhaul Brampton,Zipcar,Zoom Rent A Car
0,"West Applewood, West Dixie, NW Lakeview",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"West Applewood, West Dixie, NW Lakeview",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"West Applewood, West Dixie, NW Lakeview",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"West Applewood, West Dixie, NW Lakeview",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,"West Applewood, West Dixie, NW Lakeview",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


#### Group by neighborhood

In [71]:
miss_grouped = miss_onehot.groupby('Neighborhood').mean().reset_index()
miss_grouped

,Neighborhood,Alamo Car Rental,Alamo Rent A Car,Auto Loan Solutions,Avis Car Rental,Budget Car & Truck Rental,Budget Car Rental (Budget Car & Truck Rental),Budget Truck Rental,Denny's,Discount Car & Truck Rentals,Discount Car and Truck Rentals,Dollar,Dollar Car Rental,Dollar Rent A Car,E-Z Rent-A-Car,Enterprise Rent-A-Car,Enterprise Rent-a-Car,Hertz,Hertz Car Rental,Kal Tire,National Car Rental,Payless Car Rental,Routes Car & Truck Rental,Serena Motors Ltd,Thrifty Car Rental,Toronto Airport Limo Link,U-Haul Moving & Storage at Rexdale Blvd,U-Haul at Weston Rd,Uhaul Brampton,Zipcar,Zoom Rent A Car
0,Central Lakeview,0.000000,0.000000,0.156250,0.125000,0.000000,0.093750,0.031250,0.031250,0.062500,0.000000,0.000000,0.000000,0.062500,0.000000,0.218750,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.062500,0.000000,0.000000,0.000000,0.000000,0.062500,0.031250
1,"Churchill Meadows, Central Erin Mills, South S...",0.000000,0.000000,0.166667,0.125000,0.000000,0.125000,0.041667,0.000000,0.041667,0.083333,0.000000,0.000000,0.041667,0.000000,0.250000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Clarkson,0.000000,0.000000,0.142857,0.142857,0.000000,0.142857,0.047619,0.000000,0.095238,0.000000,0.000000,0.000000,0.047619,0.000000,0.238095,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Clarkson, Erin Mills",0.000000,0.000000,0.153846,0.153846,0.000000,0.153846,0.038462,0.000000,0.038462,0.038462,0.000000,0.000000,0.038462,0.000000,0.269231,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Courtney Park, East Gateway",0.016393,0.016393,0.065574,0.098361,0.016393,0.081967,0.000000,0.016393,0.065574,0.016393,0.016393,0.016393,0.049180,0.016393,0.213115,0.016393,0.049180,0.016393,0.016393,0.016393,0.032787,0.016393,0.016393,0.049180,0.016393,0.016393,0.000000,0.016393,0.000000,0.016393
5,"East Applewood, East Dixie, NE Lakeview",0.018868,0.018868,0.094340,0.094340,0.018868,0.056604,0.000000,0.018868,0.056604,0.000000,0.018868,0.018868,0.056604,0.018868,0.207547,0.018868,0.037736,0.018868,0.000000,0.018868,0.037736,0.000000,0.018868,0.056604,0.000000,0.000000,0.000000,0.000000,0.075472,0.018868
6,East Credit,0.000000,0.000000,0.137931,0.103448,0.000000,0.103448,0.000000,0.034483,0.068966,0.068966,0.000000,0.000000,0.068966,0.000000,0.241379,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.068966,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483
7,Erin Mills,0.000000,0.000000,0.148148,0.148148,0.000000,0.148148,0.037037,0.000000,0.037037,0.074074,0.000000,0.000000,0.037037,0.000000,0.259259,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Etobicoke, Malton, Streetsville, Toronto Pears...",0.023256,0.023256,0.023256,0.093023,0.023256,0.069767,0.000000,0.023256,0.093023,0.000000,0.023256,0.023256,0.023256,0.023256,0.186047,0.023256,0.046512,0.023256,0.046512,0.023256,0.046512,0.023256,0.000000,0.023256,0.023256,0.023256,0.000000,0.023256,0.000000,0.023256
9,"Etobicoke, Malton, Toronto Pearson Internation...",0.023256,0.023256,0.023256,0.093023,0.023256,0.069767,0.000000,0.023256,0.093023,0.000000,0.023256,0.023256,0.023256,0.023256,0.186047,0.023256,0.046512,0.023256,0.046512,0.023256,0.046512,0.023256,0.000000,0.023256,0.023256,0.023256,0.000000,0.023256,0.000000,0.023256


In [72]:
num_top_venues = 5

for hood in miss_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = miss_grouped[miss_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Lakeview----
                                           venue  freq
0                          Enterprise Rent-A-Car  0.22
1                            Auto Loan Solutions  0.16
2                                Avis Car Rental  0.12
3  Budget Car Rental (Budget Car & Truck Rental)  0.09
4                                         Zipcar  0.06


----Churchill Meadows, Central Erin Mills, South Streetsville----
                                           venue  freq
0                          Enterprise Rent-A-Car  0.25
1                            Auto Loan Solutions  0.17
2                                Avis Car Rental  0.12
3  Budget Car Rental (Budget Car & Truck Rental)  0.12
4                 Discount Car and Truck Rentals  0.08


----Clarkson----
                                           venue  freq
0                          Enterprise Rent-A-Car  0.24
1                            Auto Loan Solutions  0.14
2                                Avis Car Rental  0.14
3  Budge

#### Top Rental car venues in each neighborhood

In [73]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = miss_grouped['Neighborhood']

for ind in np.arange(miss_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(miss_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Lakeview,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Thrifty Car Rental,Discount Car & Truck Rentals,Dollar Rent A Car,Zipcar,Hertz,Budget Truck Rental
1,"Churchill Meadows, Central Erin Mills, South S...",Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car and Truck Rentals,Dollar Rent A Car,Thrifty Car Rental,Serena Motors Ltd,Budget Truck Rental,Discount Car & Truck Rentals
2,Clarkson,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car & Truck Rentals,Dollar Rent A Car,Thrifty Car Rental,Serena Motors Ltd,Budget Truck Rental,Hertz
3,"Clarkson, Erin Mills",Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Dollar Rent A Car,Thrifty Car Rental,Serena Motors Ltd,Budget Truck Rental,Discount Car & Truck Rentals,Discount Car and Truck Rentals
4,"Courtney Park, East Gateway",Enterprise Rent-A-Car,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Auto Loan Solutions,Discount Car & Truck Rentals,Thrifty Car Rental,Hertz,Dollar Rent A Car,Payless Car Rental,E-Z Rent-A-Car


In [75]:
venues1_df_new = venues1_df.drop(['Venue Name'], axis=1)
venues1_df_new.head()

,Neighborhood,Latitude,Longitude,Venue Latitude,Venue Longitude,Venue Category
0,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,43.683041,-79.611507,Rental Car Location
1,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,43.682373,-79.611094,Rental Car Location
2,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,43.683010,-79.612779,Rental Car Location
3,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,43.686466,-79.619245,Rental Car Location
4,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,43.682549,-79.612010,Rental Car Location


#### Clustering Model for the Rental car venue 

In [76]:
# set number of clusters
kclusters = 4

X= miss_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(X)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 1, 3, 0, 0, 1, 3, 2, 2], dtype=int32)

In [77]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

miss_merged = venues1_df

# merge scarborough_grouped with scarborough_data to add latitude/longitude for each neighborhood
miss_merged = miss_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

miss_merged.head()

,Neighborhood,Latitude,Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,Enterprise Rent-A-Car,43.683041,-79.611507,Rental Car Location,0,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Thrifty Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car & Truck Rentals,Dollar Rent A Car,Payless Car Rental,Hertz,E-Z Rent-A-Car
1,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,Hertz,43.682373,-79.611094,Rental Car Location,0,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Thrifty Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car & Truck Rentals,Dollar Rent A Car,Payless Car Rental,Hertz,E-Z Rent-A-Car
2,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,Avis Car Rental,43.683010,-79.612779,Rental Car Location,0,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Thrifty Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car & Truck Rentals,Dollar Rent A Car,Payless Car Rental,Hertz,E-Z Rent-A-Car
3,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,National Car Rental,43.686466,-79.619245,Rental Car Location,0,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Thrifty Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car & Truck Rentals,Dollar Rent A Car,Payless Car Rental,Hertz,E-Z Rent-A-Car
4,"West Applewood, West Dixie, NW Lakeview",43.6076,-79.5997,Hertz Car Rental,43.682549,-79.612010,Rental Car Location,0,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Thrifty Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car & Truck Rentals,Dollar Rent A Car,Payless Car Rental,Hertz,E-Z Rent-A-Car


#### Cluster map - Rental companies association by Venue name

In [78]:
# create map
map_clusters_comp = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(miss_merged['Latitude'], miss_merged['Longitude'], miss_merged['Neighborhood'], miss_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow [cluster-1],
        fill=True,
        fill_color=rainbow [cluster-1],
        fill_opacity=0.7).add_to(map_clusters_comp)
       
map_clusters_comp

In [79]:
Cluster_00 = miss_merged.loc[miss_merged['Cluster Labels'] == 0, miss_merged.columns[[0] + list(range(4, miss_merged.shape[1]))]]
Cluster_00.head() #Red - Enterprise, Auto Loan solutions, Avis, Thrifty Car, Budget...Hertz and Zoom Rent A car in last.


,Neighborhood,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"West Applewood, West Dixie, NW Lakeview",43.683041,-79.611507,Rental Car Location,0,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Thrifty Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car & Truck Rentals,Dollar Rent A Car,Payless Car Rental,Hertz,E-Z Rent-A-Car
1,"West Applewood, West Dixie, NW Lakeview",43.682373,-79.611094,Rental Car Location,0,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Thrifty Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car & Truck Rentals,Dollar Rent A Car,Payless Car Rental,Hertz,E-Z Rent-A-Car
2,"West Applewood, West Dixie, NW Lakeview",43.683010,-79.612779,Rental Car Location,0,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Thrifty Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car & Truck Rentals,Dollar Rent A Car,Payless Car Rental,Hertz,E-Z Rent-A-Car
3,"West Applewood, West Dixie, NW Lakeview",43.686466,-79.619245,Rental Car Location,0,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Thrifty Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car & Truck Rentals,Dollar Rent A Car,Payless Car Rental,Hertz,E-Z Rent-A-Car
4,"West Applewood, West Dixie, NW Lakeview",43.682549,-79.612010,Rental Car Location,0,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Thrifty Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car & Truck Rentals,Dollar Rent A Car,Payless Car Rental,Hertz,E-Z Rent-A-Car


In [80]:
Cluster_11 = miss_merged.loc[miss_merged['Cluster Labels'] == 1, miss_merged.columns[[0] + list(range(4, miss_merged.shape[1]))]]
Cluster_11.head() #Purple - Enterprise, Auto loan, Avis, Budget car, Discount car on fifth, Hertz is least



,Neighborhood,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
126,"Mississauga Valleys, East Cooksville",43.588777,-79.607382,Rental Car Location,1,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car & Truck Rentals,Thrifty Car Rental,Dollar Rent A Car,Hertz,Budget Truck Rental,Denny's
127,"Mississauga Valleys, East Cooksville",43.586025,-79.608886,Rental Car Location,1,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car & Truck Rentals,Thrifty Car Rental,Dollar Rent A Car,Hertz,Budget Truck Rental,Denny's
128,"Mississauga Valleys, East Cooksville",43.584694,-79.611418,Rental Car Location,1,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car & Truck Rentals,Thrifty Car Rental,Dollar Rent A Car,Hertz,Budget Truck Rental,Denny's
129,"Mississauga Valleys, East Cooksville",43.592308,-79.633863,Rental Car Location,1,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car & Truck Rentals,Thrifty Car Rental,Dollar Rent A Car,Hertz,Budget Truck Rental,Denny's
130,"Mississauga Valleys, East Cooksville",43.592470,-79.634890,Rental Car Location,1,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car & Truck Rentals,Thrifty Car Rental,Dollar Rent A Car,Hertz,Budget Truck Rental,Denny's


In [81]:
Cluster_22 = miss_merged.loc[miss_merged['Cluster Labels'] == 2, miss_merged.columns[[0] + list(range(4, miss_merged.shape[1]))]]
Cluster_22.head() # Blue - Enterprise, Auto loan on 2nd, Avis, Budget, Discount on fifth..... Hertz is last
### Its near Airport

,Neighborhood,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
593,Malton,43.683041,-79.611507,Rental Car Location,2,Enterprise Rent-A-Car,Discount Car & Truck Rentals,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Hertz,Payless Car Rental,Kal Tire,E-Z Rent-A-Car,Alamo Rent A Car,Auto Loan Solutions
594,Malton,43.682373,-79.611094,Rental Car Location,2,Enterprise Rent-A-Car,Discount Car & Truck Rentals,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Hertz,Payless Car Rental,Kal Tire,E-Z Rent-A-Car,Alamo Rent A Car,Auto Loan Solutions
595,Malton,43.683010,-79.612779,Rental Car Location,2,Enterprise Rent-A-Car,Discount Car & Truck Rentals,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Hertz,Payless Car Rental,Kal Tire,E-Z Rent-A-Car,Alamo Rent A Car,Auto Loan Solutions
596,Malton,43.686466,-79.619245,Rental Car Location,2,Enterprise Rent-A-Car,Discount Car & Truck Rentals,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Hertz,Payless Car Rental,Kal Tire,E-Z Rent-A-Car,Alamo Rent A Car,Auto Loan Solutions
597,Malton,43.716474,-79.625551,Rental Car Location,2,Enterprise Rent-A-Car,Discount Car & Truck Rentals,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Hertz,Payless Car Rental,Kal Tire,E-Z Rent-A-Car,Alamo Rent A Car,Auto Loan Solutions


In [82]:
Cluster_33 = miss_merged.loc[miss_merged['Cluster Labels'] == 3, miss_merged.columns[[0] + list(range(4, miss_merged.shape[1]))]]
Cluster_33.head() #green - Enterprise, discount, Avis, Budget, Hertz....Auto loan and Budget car and truck is last

,Neighborhood,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,"Churchill Meadows, Central Erin Mills, South S...",43.585101,-79.741007,Rental Car Location,3,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car and Truck Rentals,Dollar Rent A Car,Thrifty Car Rental,Serena Motors Ltd,Budget Truck Rental,Discount Car & Truck Rentals
50,"Churchill Meadows, Central Erin Mills, South S...",43.589062,-79.739822,Rental Car Location,3,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car and Truck Rentals,Dollar Rent A Car,Thrifty Car Rental,Serena Motors Ltd,Budget Truck Rental,Discount Car & Truck Rentals
51,"Churchill Meadows, Central Erin Mills, South S...",43.592556,-79.727840,Rental Car Location,3,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car and Truck Rentals,Dollar Rent A Car,Thrifty Car Rental,Serena Motors Ltd,Budget Truck Rental,Discount Car & Truck Rentals
52,"Churchill Meadows, Central Erin Mills, South S...",43.591560,-79.741431,Rental Car Location,3,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car and Truck Rentals,Dollar Rent A Car,Thrifty Car Rental,Serena Motors Ltd,Budget Truck Rental,Discount Car & Truck Rentals
53,"Churchill Meadows, Central Erin Mills, South S...",43.595236,-79.731626,Rental Car Location,3,Enterprise Rent-A-Car,Auto Loan Solutions,Avis Car Rental,Budget Car Rental (Budget Car & Truck Rental),Discount Car and Truck Rentals,Dollar Rent A Car,Thrifty Car Rental,Serena Motors Ltd,Budget Truck Rental,Discount Car & Truck Rentals
